In [50]:
import pandas as pd
import sys
path = '/Users/llupo/dev/pappa'
if not path in sys.path :
    sys.path.append(path)
from lm_classifiers import GPTClassifier
from task_manager import TaskManager


In [57]:
# import tsv as pandas
df = pd.read_csv('../results/pappa/alldim/long_fewshot_gpt-4/pred_dim1.tsv', sep='\t', index_col=0)
df_labels_only = df[['prediction', 'gold_elin', 'gold_lena', 'gold_oscar']].fillna('NA')
# save new df as csv without column names
df_labels_only.to_csv('../results/pappa/alldim/long_fewshot_gpt-4/pred_dim1_labelsonly.csv', index=False, sep=',', header=False)


In [58]:
! /Users/llupo/dev/MACE/MACE --prefix ../results/pappa/alldim/long_fewshot_gpt-4/pred_dim1_mace ../results/pappa/alldim/long_fewshot_gpt-4/pred_dim1_labelsonly.csv

Reading CSV file '../results/pappa/alldim/long_fewshot_gpt-4/pred_dim1_labelsonly.csv'
....................100
....................200
....................300
.........
stats:
	350 instances,
	6 labels [PASSIVE, NA, ACTIVE_POS_OTHER, ACTIVE_NEG, ACTIVE_POS_CHALLENGING, ACTIVE_POS_CARING],
	4 annotators

Running Variational Bayes EM training with the following settings:
	50 iterations
	10 restarts
	smoothing = 0.0016666666666666668
	alpha = 0.5
	beta = 0.5

Restart 1
initial log marginal likelihood = -1895.7061527654348
final log marginal likelihood = -1655.643281959987

Restart 2
initial log marginal likelihood = -1933.3593233684114
final log marginal likelihood = -1655.6432819610707

Restart 3
initial log marginal likelihood = -1878.6215103738234
final log marginal likelihood = -1655.6432819590148

Restart 4
initial log marginal likelihood = -1928.651756193378
final log marginal likelihood = -1655.6432819592212

Restart 5
initial log marginal likelihood = -1903.878022857036
final log 

In [59]:
df_mace_pred = pd.read_csv('../results/pappa/alldim/long_fewshot_gpt-4/pred_dim1_mace.prediction', header=None)
df_mace_pred.rename(columns={0: 'gold_mace2'}, inplace=True)
# join df_mace_pred to df
df = df.join(df_mace_pred, how='left')
df

,prompt,prediction,gold_elin,gold_lena,gold_oscar,gold_agg,gold_mace2
0,NaN,PASSIVE,PASSIVE,PASSIVE,PASSIVE,PASSIVE,PASSIVE
1,NaN,PASSIVE,PASSIVE,NaN,NaN,NaN,PASSIVE
2,NaN,PASSIVE,ACTIVE_POS_OTHER,ACTIVE_POS_OTHER,PASSIVE,ACTIVE_POS_OTHER,ACTIVE_POS_OTHER
3,NaN,ACTIVE_POS_OTHER,ACTIVE_POS_OTHER,ACTIVE_POS_OTHER,ACTIVE_POS_OTHER,ACTIVE_POS_OTHER,ACTIVE_POS_OTHER
4,NaN,PASSIVE,PASSIVE,PASSIVE,PASSIVE,PASSIVE,PASSIVE
...,...,...,...,...,...,...,...
345,NaN,ACTIVE_NEG,ACTIVE_NEG,ACTIVE_NEG,ACTIVE_NEG,ACTIVE_NEG,ACTIVE_NEG
346,NaN,NaN,NaN,NaN,NaN,NaN,NaN
347,NaN,PASSIVE,NaN,PASSIVE,PASSIVE,PASSIVE,PASSIVE
348,NaN,ACTIVE_POS_OTHER,ACTIVE_POS_OTHER,ACTIVE_POS_CARING,ACTIVE_POS_CARING,ACTIVE_POS_CARING,ACTIVE_POS_CARING


In [60]:

# model_name=gpt-4
# max_len_model=4097
#     python main.py \
#     --data_file data/human_annotation/dim3.csv \
#     --instruction instructions/$instruction.txt \
#     --output_prompt "\\nLabel: " \
#     --model_name $model_name \
#     --max_len_model $max_len_model \
#     --task pappa_dim3 \
#     --output_dir tmp \
#     --evaluation_only \
#     --only_dim 2

task = 'pappa_dim1'
data_file = '../data/human_annotation/dim1.csv'

tm = TaskManager(task)
input_texts, _ = tm.read_data(data_file)
# gold labels are columns starting with gold_
gold_labels = df[[col for col in df.columns if col.startswith('gold_')]].fillna('NA')
gold_labels.columns = [col.replace('gold_', '') for col in gold_labels.columns]
classifier = GPTClassifier(input_texts, tm.labels, tm.label_dims)
df_kappa, df_accuracy, df_f1 = classifier.evaluate_predictions(df=df, gold_labels=gold_labels)

Evaluating predicitons...
   prompt        prediction         gold_elin         gold_lena  \
0     NaN           PASSIVE           PASSIVE           PASSIVE   
1     NaN           PASSIVE           PASSIVE                NA   
2     NaN           PASSIVE  ACTIVE_POS_OTHER  ACTIVE_POS_OTHER   
3     NaN  ACTIVE_POS_OTHER  ACTIVE_POS_OTHER  ACTIVE_POS_OTHER   
4     NaN           PASSIVE           PASSIVE           PASSIVE   

         gold_oscar          gold_agg        gold_mace2  
0           PASSIVE           PASSIVE           PASSIVE  
1                NA                NA           PASSIVE  
2           PASSIVE  ACTIVE_POS_OTHER  ACTIVE_POS_OTHER  
3  ACTIVE_POS_OTHER  ACTIVE_POS_OTHER  ACTIVE_POS_OTHER  
4           PASSIVE           PASSIVE           PASSIVE  
KAPPA:
         elin    lena   oscar     agg   mace2   model  mean_non_agg
elin   100.00   48.68   49.93   67.28   71.18   39.24         56.60
lena    48.68  100.00   53.37   75.09   69.09   46.90         57.05
oscar   49.9

In [53]:
df[[col for col in df.columns if col.startswith('gold_')]]
# remove the "gold_" prefix from the column names
df.columns = [col.replace('gold_', '') for col in df.columns]

,gold_elin,gold_lena,gold_oscar,gold_agg
0,PASSIVE,PASSIVE,PASSIVE,PASSIVE
1,PASSIVE,NaN,NaN,NaN
2,ACTIVE_POS_OTHER,ACTIVE_POS_OTHER,PASSIVE,ACTIVE_POS_OTHER
3,ACTIVE_POS_OTHER,ACTIVE_POS_OTHER,ACTIVE_POS_OTHER,ACTIVE_POS_OTHER
4,PASSIVE,PASSIVE,PASSIVE,PASSIVE
...,...,...,...,...
345,ACTIVE_NEG,ACTIVE_NEG,ACTIVE_NEG,ACTIVE_NEG
346,NaN,NaN,NaN,NaN
347,NaN,PASSIVE,PASSIVE,PASSIVE
348,ACTIVE_POS_OTHER,ACTIVE_POS_CARING,ACTIVE_POS_CARING,ACTIVE_POS_CARING
